In [5]:
import box_lib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from IPython.core.debugger import set_trace
import numpy as np



def main():
  np.random.seed(89)
  torch.manual_seed(43)

  train_ds = box_lib.BoxDataset("data/sample/train.txt")
  train_dl = DataLoader(train_ds, batch_size=18, shuffle=True, num_workers=4)

  model = box_lib.Boxes(6,4)
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


  N_EPOCHS = 200

  for epoch in range(N_EPOCHS):
      
      # Train
      model.train()  # IMPORTANT
      
      running_loss, correct = 0.0, 0
      for X, y in train_dl:
        #set_trace()
        #X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
          y_ = model(X)
          loss = criterion(y_, y)
        optimizer.step()
        
        # Statistics
        print("    batch loss: "+str(loss.item()))
        running_loss += loss.item() * X.shape[0]

      print("  Train Loss: "+str(running_loss / len(train_dl.dataset)))
      print("  Train Acc:  "+str(correct / len(train_dl.dataset)))
      

if __name__ == "__main__":
  main()


torch.Size([18, 2])
tensor([ -0.0065, -12.1258,  -0.1219,   0.0242,   0.0090,  14.2781,  -0.0435,
         -0.0184,   0.0275,  -0.0439,   0.9767,  -0.0038,   3.6613,  -0.0019,
         -6.7041,   0.0374,   0.0036,  -0.0017], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0019,  -0.0017,  -0.0435,  -0.0439,  -0.0038,  -0.1219,  -0.0065,
         -6.7041,   0.0374,   0.0275,  14.2781,   0.0242,   0.9767,   3.6613,
         -0.0184, -12.1258,   0.0090,   0.0036], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -6.7041,   0.0275,  -0.0439,  -0.0184,   3.6613,   0.9767,  14.2781,
         -0.0065,  -0.0017,  -0.0435,  -0.0038,   0.0090,   0.0036,  -0.1219,
         -0.0019,   0.0242, -12.1258,   0.0374], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0019,   0.0242,   0.0036,  -0.0439,  14.2781,  -0.1219,  -0

tensor([  0.9767,  -0.0019,  -0.0439,   0.0374,   0.0036,  14.2781,   0.0275,
          0.0090,  -0.1219,  -0.0038,   0.0242,  -0.0435,   3.6613,  -0.0065,
         -0.0017,  -6.7041,  -0.0184, -12.1258], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.9767,  -0.0184,   0.0036,   0.0090,  -0.0017,  -0.0038,   0.0242,
        -12.1258,  -6.7041,  -0.0065,  -0.0019,   3.6613,   0.0275,  -0.0439,
          0.0374,  -0.0435,  14.2781,  -0.1219], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.0374,  -0.0065,  14.2781,  -0.0017,   0.9767, -12.1258,  -0.0019,
         -0.0038,  -0.1219,  -6.7041,   0.0090,  -0.0439,  -0.0184,   0.0242,
          3.6613,   0.0275,  -0.0435,   0.0036], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -6.7041,   0.0036,   0.0275,   3.6613,  -0.0435,  -0.0019,  -0.0184,
          0.0

tensor([ -0.0038,  -0.0017, -12.1258,  -0.1219,   0.0374,  14.2781,   0.0090,
          0.0242,  -0.0435,  -0.0439,   0.0275,  -0.0065,   0.9767,   3.6613,
          0.0036,  -0.0184,  -0.0019,  -6.7041], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0017,   0.0036,   0.0275,   0.9767,  -0.0019,  -6.7041,  -0.0038,
         -0.1219,  -0.0065,   0.0242, -12.1258,   3.6613,  -0.0435,   0.0374,
         -0.0184,  -0.0439,  14.2781,   0.0090], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0065,   0.0275,   0.0242,  14.2781,  -0.0019,  -0.0435,  -6.7041,
         -0.0038,   3.6613, -12.1258,  -0.0439,   0.0090,  -0.1219,   0.0374,
          0.9767,   0.0036,  -0.0017,  -0.0184], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.0374,  -0.0019,  -6.7041,   3.6613,  -0.0038,   0.0242,  -0.0439,
         14.2

tensor([ -0.0439, -12.1258,  14.2781,  -6.7041,   0.0275,   3.6613,   0.0036,
         -0.0019,   0.0090,   0.9767,   0.0242,  -0.0038,  -0.0017,  -0.1219,
          0.0374,  -0.0065,  -0.0435,  -0.0184], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0019,  -0.0017,   0.0275,  -6.7041,  -0.0065,  -0.0038,   0.0374,
         -0.1219,  14.2781,  -0.0439,   0.9767, -12.1258,   0.0242,   3.6613,
          0.0036,   0.0090,  -0.0184,  -0.0435], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.1219,  -0.0017,  14.2781, -12.1258,   0.0090,  -0.0019,  -0.0038,
          0.0036,   0.0275,  -0.0435,   0.0242,   3.6613,  -0.0439,  -0.0184,
         -0.0065,   0.0374,  -6.7041,   0.9767], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0435,  -0.0019,  -0.0065,  -0.0184, -12.1258,   3.6613,   0.0036,
          0.0

tensor([  0.0374,   0.0242,  -0.0184,   0.0036,   3.6613, -12.1258,  -0.0038,
         -0.0439,  -6.7041,  -0.1219,   0.0275,  -0.0017,  -0.0435,   0.9767,
         -0.0019,  14.2781,   0.0090,  -0.0065], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.0036,  -0.0019,   3.6613,  -0.0435,   0.0242,   0.0275,  -0.0184,
        -12.1258,   0.9767,  -0.1219,   0.0374,  -6.7041,   0.0090,  14.2781,
         -0.0439,  -0.0038,  -0.0065,  -0.0017], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0038,   0.0275,  -0.0435, -12.1258,  -6.7041,  -0.1219,  -0.0017,
          0.9767,   0.0036,  -0.0065,   0.0374,  -0.0184,  -0.0439,   0.0242,
         -0.0019,   0.0090,  14.2781,   3.6613], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.9767,   0.0242,  -0.0439,  -0.0038,   0.0275,   3.6613,  -0.0065,
         -0.0

tensor([  0.9767,  -0.0017,   0.0242,  -0.0065,   3.6613,  -0.0019,  -0.0435,
          0.0275,  14.2781,   0.0374,  -0.0038,  -6.7041,  -0.0439, -12.1258,
          0.0036,  -0.1219,   0.0090,  -0.0184], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.0275, -12.1258,  -0.0439,  14.2781,   3.6613,  -0.0065,   0.0242,
         -0.0435,  -6.7041,   0.0374,  -0.0017,  -0.0019,   0.0036,   0.0090,
         -0.0184,   0.9767,  -0.0038,  -0.1219], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.0374,   0.0275,  -0.0017,   0.0036,   3.6613,  -0.0435,   0.0090,
        -12.1258,  14.2781,  -0.0184,  -6.7041,  -0.0439,   0.0242,  -0.0038,
         -0.0019,  -0.0065,   0.9767,  -0.1219], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0439,   0.0090,  14.2781,   0.0036,   0.0275,  -0.0435,  -0.0184,
          0.0

tensor([ -0.0038,   0.0374,   0.0275,  -0.0065,  -0.0439,   0.0036,  -0.0019,
          0.9767,   3.6613,  -0.0435,   0.0090,  -0.0017,  14.2781,  -0.0184,
         -6.7041, -12.1258,  -0.1219,   0.0242], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0435,   0.0275,  -0.0439, -12.1258,  -0.0184,  -0.0038,   0.0374,
         -0.0065,  -0.1219,   0.9767,   0.0242,  14.2781,   0.0090,  -0.0017,
          0.0036,   3.6613,  -0.0019,  -6.7041], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ 14.2781,   0.0374,  -0.0065, -12.1258,  -0.0184,  -0.0038,   0.0036,
          0.0090,   0.0242,   0.9767,  -0.1219,   0.0275,  -0.0435,  -0.0017,
         -0.0019,  -6.7041,  -0.0439,   3.6613], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([  0.0242,  -0.0017, -12.1258,  -0.0065,  -0.0184,   0.0275,   0.9767,
         -0.1

tensor([  0.0374,  -0.0184,  -6.7041,   0.9767,  -0.0065,  14.2781,  -0.1219,
          0.0275,   0.0090, -12.1258,  -0.0038,   3.6613,  -0.0017,  -0.0439,
         -0.0019,   0.0036,   0.0242,  -0.0435], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.1219,  -0.0439,  -0.0065,  -6.7041,   0.0374,  14.2781,   0.0036,
         -0.0017,  -0.0038,   3.6613,  -0.0019,   0.0242,   0.9767, -12.1258,
         -0.0435,  -0.0184,   0.0275,   0.0090], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0065,   0.0242,   0.0275,  -0.1219,  14.2781,  -0.0019,   0.0374,
        -12.1258,  -0.0184,  -0.0017,   0.0090,  -6.7041,   3.6613,   0.9767,
         -0.0439,  -0.0435,   0.0036,  -0.0038], grad_fn=<DivBackward1>)
    batch loss: 22.2278575897
  Train Loss: 22.2278575897
  Train Acc:  0
tensor([ -0.0065,  -0.0017,   0.0374,  -0.0038,  -6.7041,   0.0036,   0.9767,
          0.0